# Find spike days

## Load all companies

In [79]:
import numpy as np
import pandas as pd

In [3]:
from db_operation import DBService

db = DBService()

db.connect()

companies = db.get_companies()

companies.head()

,symbol,name,ipo_year,sector,industry
0,AAPC,Atlantic Alliance Partnership Corp.,2015,Consumer Services,Real Estate Investment Trusts
1,ABTL,Autobytel Inc.,1999,Technology,"Computer Software: Programming, Data Processing"
2,ACCP,"Accelerated Pharma, Inc.",0,n/a,n/a
3,ABIO,"ARCA biopharma, Inc.",0,Health Care,Biotechnology: In Vitro & In Vivo Diagnostic S...
4,ABMD,"ABIOMED, Inc.",0,Health Care,Medical/Dental Instruments


## Get all quotes

In [52]:
#< symbol, quote list>
all_quotes = {}

In [ ]:
for symbol in companies.symbol.values:
    if symbol in all_quotes:
        quotes = all_quotes[symbol]
    else:
        quotes = db.get_quotes_by_symbol(symbol)
        if np.any(quotes['volume'] < 100):
            print('Zero volume:', symbol)
            continue
        else:
            all_quotes[symbol] = quotes

print('Load success:', len(all_quotes))
all_quotes['SNAP'].head()

## Reports


In [97]:
reports = []
target_dt =  -1
period = 300
for sector, group in companies[['symbol', 'sector']].groupby(['sector']):
    
    up_count = 0
    down_count = 0
    for i in range(period):
        cur_dt = target_dt - i
        
        for symbol in group['symbol']:
            if symbol in all_quotes and len(all_quotes[symbol]) > period:
                quote_dt = all_quotes[symbol]['quote_date'].iloc[cur_dt]
                close_p = all_quotes[symbol]['close']

                if close_p.iloc[cur_dt] > close_p.iloc[cur_dt - 1]:
                    up_count += 1
                else:
                    down_count += 1
        reports.append([quote_dt ,sector, up_count, down_count, up_count + down_count, up_count / (up_count + down_count)])

reports = pd.DataFrame(reports, columns = ['quote_dt', 'sector', 'up_count', 'down_count', 'total', 'prec'])
print(reports)
print(reports['total'].sum(), reports['up_count'].sum(), reports['down_count'].sum())

      quote_dt            sector  up_count  down_count   total      prec
0     20171113  Basic Industries        93         119     212  0.438679
1     20171110  Basic Industries       176         248     424  0.415094
2     20171109  Basic Industries       234         402     636  0.367925
3     20171108  Basic Industries       335         513     848  0.395047
4     20171107  Basic Industries       400         660    1060  0.377358
5     20171106  Basic Industries       527         745    1272  0.414308
6     20171103  Basic Industries       617         867    1484  0.415768
7     20171102  Basic Industries       713         983    1696  0.420401
8     20171101  Basic Industries       818        1090    1908  0.428721
9     20171031  Basic Industries       940        1180    2120  0.443396
10    20171030  Basic Industries      1008        1324    2332  0.432247
11    20171027  Basic Industries      1113        1431    2544  0.437500
12    20171026  Basic Industries      1230        1

,sector,up_count,down_count,total
0,Basic Industries,97,127,224
1,Capital Goods,116,150,266
2,Consumer Durables,56,36,92
3,Consumer Non-Durables,75,82,157
4,Consumer Services,304,323,627
5,Energy,46,193,239
6,Finance,311,192,503
7,Health Care,220,234,454
8,Miscellaneous,43,56,99
9,Public Utilities,96,103,199


## Find spike

In [62]:
%%time

count = 0
for symbol, quotes in all_quotes.items():

    if len(quotes) < 150:
        continue
    
    target_dt = -100
    sma_period = 20
    sma_vol50 = np.mean(quotes['volume'].iloc[target_dt - sma_period: target_dt])
    vol = quotes['volume'].iloc[target_dt]
    if vol > sma_vol50 * 2:
        count += 1
        print(count, '\t', symbol, '\t', quotes['quote_date'].iloc[target_dt], sma_vol50, vol, int(vol / sma_vol50))



1 	 BKMU 	 20170623 90652.4 252543 2
2 	 CSFL 	 20170623 296272.65 789875 2
3 	 XCO 	 20170623 161620.35 1488561 9
4 	 BUFF 	 20170623 1204211.75 4238386 3
5 	 FOGO 	 20170623 118130.6 440276 3
6 	 ADC 	 20170623 292565.85 788841 2
7 	 SBCF 	 20170623 321825.35 1291089 4
8 	 ESS 	 20170623 316999.45 783872 2
9 	 AVEO 	 20170623 611312.05 54863640 89
10 	 LECO 	 20170623 349087.85 1299664 3
11 	 SALT 	 20170623 606064.35 1242513 2
12 	 YUMC 	 20170623 3402010.6 7244558 2
13 	 FFG 	 20170623 18124.25 45449 2
14 	 TTI 	 20170623 1004391.85 4687212 4
15 	 CCRN 	 20170623 318652.1 843705 2
16 	 QNST 	 20170623 68190.9 426380 6
17 	 STAR 	 20170623 439259.55 1846226 4
18 	 COBZ 	 20170623 78073.6 379856 4
19 	 BUSE 	 20170623 104792.65 307455 2
20 	 BCEI 	 20170623 185698.6 2615762 14
21 	 BANF 	 20170623 54375.8 200132 3
22 	 BPOP 	 20170623 472262.2 1019443 2
23 	 ARGS 	 20170623 595057.7 2442858 4
24 	 PBPB 	 20170623 374092.75 1081424 2
25 	 COR 	 20170623 388701.45 3697469 9
26 	 FMBI 	

In [51]:
np.any( all_quotes['SNAP']['volume'] > 110)

True

,quote_date,open,close,high,low,volume
0,20170302,24.00,24.48,26.05,23.50,216978300
1,20170303,26.39,27.09,29.44,26.06,148099300
2,20170306,28.17,23.77,28.25,23.77,72804120
3,20170307,22.21,21.44,22.50,20.64,71810660
4,20170308,22.03,22.81,23.43,21.31,49784130
